# Code that grabs all the index data

In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
import statsmodels.api as sm
import numpy as np

# Define the indexes and their inception dates
indexes = ['SPY', 'VTI', 'IVV', 'QQQ', 'VXUS', 'IJH', 'IJR', 'VEA', 'VWO', 'DIA']
inception_dates = ['1993-1-24', '2001-5-25', '2000-5-16', '1999-3-10', '2011-1-27',
                   '2000-5-23', '2000-5-23', '2007-7-21', '2005-3-5', '1998-1-15']
df = pd.DataFrame({'Index': indexes, 'Inception_Date': inception_dates})

# Function to get all the monthly returns
def get_monthly_returns(df):
    all_returns = []
    for _, row in df.iterrows():
        ticker = row['Index']
        inception_date = row['Inception_Date']
        start = pd.to_datetime(inception_date)
        end = dt.datetime.today().strftime('%Y-%m-%d')
        data = yf.download(ticker, start=start, end=end)
        if not data.empty:
            monthly_prices = data['Adj Close'].resample('M').last()
            monthly_returns = monthly_prices.pct_change()
            monthly_returns = monthly_returns.dropna().reset_index()
            monthly_returns.rename(columns={'Date': 'Date', 'Adj Close': 'Monthly_Return'}, inplace=True)
            monthly_returns['Index'] = ticker
            monthly_returns['Inception_Date'] = inception_date  # Include inception date
            all_returns.append(monthly_returns)
    final_df = pd.concat(all_returns, ignore_index=True)
    return final_df

# Function to create covariance matrix
# Modify the create_covariance_matrix function to handle missing data
def create_covariance_matrix(df):
    pivoted_df = df.pivot(index='Date', columns='Index', values='Monthly_Return')
    # Optional: fill missing values if any exist, such as forward fill or interpolation
    pivoted_df = pivoted_df.interpolate()  # Example using interpolation
    covariance_matrix = pivoted_df.cov()
    return covariance_matrix


# Function to calculate financial metrics using ARIMA
def calculate_financial_metrics_arima(df):
    results_list = []
    tickers = df['Index'].unique()
    for ticker in tickers:
        ticker_data = df[df['Index'] == ticker]['Monthly_Return']
        try:
            model = sm.tsa.ARIMA(ticker_data, order=(1, 0, 0))
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=12).sum()  # Sum of 12 months forecast
        except Exception as e:
            print(f"Failed to fit ARIMA model for {ticker}: {str(e)}")
            forecast = np.nan

        monthly_std = ticker_data.std()
        annualized_std = monthly_std * np.sqrt(12)
        variance = annualized_std ** 2
        inception_date = df[df['Index'] == ticker]['Inception_Date'].iloc[0]

        results_list.append({
            'Ticker': ticker,
            'Inception_Date': inception_date,
            'Expected_Annual_Return': forecast,
            'Annualized_Std': annualized_std,
            'Variance': variance})
    results_df = pd.DataFrame(results_list)
    return results_df
    
# Execute functions to fetch and process data
resulting_df = get_monthly_returns(df)
final_metrics_df = calculate_financial_metrics_arima(resulting_df)
covariance_df = create_covariance_matrix(resulting_df)

# Output the results
print("Financial Metrics DataFrame:")
print(final_metrics_df)
print("\nCovariance Matrix:")
print(covariance_df)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupp

Financial Metrics DataFrame:
  Ticker Inception_Date  Expected_Annual_Return  Annualized_Std  Variance
0    SPY      1993-1-24                0.108189        0.149081  0.022225
1    VTI      2001-5-25                0.096447        0.155671  0.024233
2    IVV      2000-5-16                0.083852        0.152767  0.023338
3    QQQ      1999-3-10                0.119990        0.237049  0.056192
4   VXUS      2011-1-27                0.054499        0.154296  0.023807
5    IJH      2000-5-23                0.106245        0.179967  0.032388
6    IJR      2000-5-23                0.109328        0.195530  0.038232
7    VEA      2007-7-21                0.050396        0.183122  0.033534
8    VWO       2005-3-5                0.083628        0.209328  0.043818
9    DIA      1998-1-15                0.093159        0.152162  0.023153

Covariance Matrix:
Index       DIA       IJH       IJR       IVV       QQQ       SPY       VEA  \
Index                                                     

C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\dpres\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: Val

In [2]:
final_metrics_df

,Ticker,Inception_Date,Expected_Annual_Return,Annualized_Std,Variance
0,SPY,1993-1-24,0.108189,0.149081,0.022225
1,VTI,2001-5-25,0.096447,0.155671,0.024233
2,IVV,2000-5-16,0.083852,0.152767,0.023338
3,QQQ,1999-3-10,0.119990,0.237049,0.056192
4,VXUS,2011-1-27,0.054499,0.154296,0.023807
5,IJH,2000-5-23,0.106245,0.179967,0.032388
6,IJR,2000-5-23,0.109328,0.195530,0.038232
7,VEA,2007-7-21,0.050396,0.183122,0.033534
8,VWO,2005-3-5,0.083628,0.209328,0.043818
9,DIA,1998-1-15,0.093159,0.152162,0.023153


In [3]:

covariance_df

Index,DIA,IJH,IJR,IVV,QQQ,SPY,VEA,VTI,VWO,VXUS
Index,,,,,,,,,,
DIA,0.001929,0.001966,0.002011,0.001800,0.002148,0.001866,0.002026,0.001827,0.001747,0.001515
IJH,0.001966,0.002699,0.002813,0.002109,0.002660,0.002115,0.002497,0.002205,0.002338,0.001894
IJR,0.002011,0.002813,0.003186,0.002153,0.002692,0.002154,0.002529,0.002279,0.002348,0.001925
IVV,0.001800,0.002109,0.002153,0.001945,0.002537,0.001950,0.002178,0.001954,0.001946,0.001589
QQQ,0.002148,0.002660,0.002692,0.002537,0.004683,0.002535,0.002338,0.002376,0.002228,0.001699
SPY,0.001866,0.002115,0.002154,0.001950,0.002535,0.001852,0.002179,0.001956,0.001951,0.001592
VEA,0.002026,0.002497,0.002529,0.002178,0.002338,0.002179,0.002794,0.002252,0.002826,0.001985
VTI,0.001827,0.002205,0.002279,0.001954,0.002376,0.001956,0.002252,0.002019,0.002045,0.001658
VWO,0.001747,0.002338,0.002348,0.001946,0.002228,0.001951,0.002826,0.002045,0.003652,0.002050


In [4]:
import pandas as pd

tickers = ['SPY', 'VTI', 'IVV', 'QQQ', 'VXUS', 'IJH', 'IJR', 'VEA', 'VWO', 'DIA']
dividend_yields = [.0132, .0140, .0130, .0060, .0330, .0135, .0134, .0335, .0344, .0180]
df = pd.DataFrame({
    'Ticker': tickers,
    'Dividend Yield (%)': dividend_yields
})

df_merged = pd.merge(final_metrics_df, df, on='Ticker', how='inner')

df_merged['Total Expected Return (%)'] = df_merged['Dividend Yield (%)'] + df_merged['Expected_Annual_Return']

values = ['Risk Free Rate', None, 0.05209, 0, 0, 0, 0.05209] 
risk_free_rate = pd.DataFrame([values], columns=df_merged.columns)


df_merged


,Ticker,Inception_Date,Expected_Annual_Return,Annualized_Std,Variance,Dividend Yield (%),Total Expected Return (%)
0,SPY,1993-1-24,0.108189,0.149081,0.022225,0.0132,0.121389
1,VTI,2001-5-25,0.096447,0.155671,0.024233,0.0140,0.110447
2,IVV,2000-5-16,0.083852,0.152767,0.023338,0.0130,0.096852
3,QQQ,1999-3-10,0.119990,0.237049,0.056192,0.0060,0.125990
4,VXUS,2011-1-27,0.054499,0.154296,0.023807,0.0330,0.087499
5,IJH,2000-5-23,0.106245,0.179967,0.032388,0.0135,0.119745
6,IJR,2000-5-23,0.109328,0.195530,0.038232,0.0134,0.122728
7,VEA,2007-7-21,0.050396,0.183122,0.033534,0.0335,0.083896
8,VWO,2005-3-5,0.083628,0.209328,0.043818,0.0344,0.118028
9,DIA,1998-1-15,0.093159,0.152162,0.023153,0.0180,0.111159


In [5]:
df_merged = df_merged.drop('Inception_Date', axis=1)
df_merged = df_merged.drop('Expected_Annual_Return', axis=1)
df_merged = df_merged.drop('Dividend Yield (%)', axis=1)

In [6]:
# Saves data frame to folder
df_merged.to_csv('Index_Data/index_data.csv', index=False)
covariance_df.to_csv('Index_Data/cov_mat.csv', index=True)